# Single Subject Brain Volumetric Analysis 
### Beta run at local Jupyter notebook that can submit jobs remotely to the HPC
*nbourke@imperial.ac.uk March 2022* 

* Set up as a Git repo with branch on HPC, 
* Jupyter lab spawned on local machine that has HPC mounted with access to file system. 

## Overview
This notebook takes a set of scans and tells you what the volumes of grey matter, white matter and CSF are. It also lets you run a voxelwise comparison (voxel based morphometry) between the two groups (patients versus controls) to see whether the volume at each voxel is significantly different. Eg. Do patients have smaller brains (ie are they more atrophic) than controls?

The notebook uses SPM12 (UCL) to do most of the heavy lifting, and then FSL for the stats. You might find the SPM manual useful: https://www.fil.ion.ucl.ac.uk/spm/doc/manual.pdf.

Matlab dependencies by Neil Graham and Greg Scott

### TO DO:
* Take template creation out of this notebook 
    - generate templates for age bins
    - match single subject to appropriate template and translate

## Prerequisites

You will need: 

-Data stored in BIDS format  

* hpcwrapmatlab.sh
* hpcrunarrayjob.sh
* segment_t1.m
* make_template.m
* generate_flowfields.m
* move_to_mni.m

**All of these scripts should be in the dependencies folder now**

### Development notes:

- Auth keys are disabled on cluster - requires passing shh details 
- Requires mounted volume (mac = osfuse)
- Need to set paths from both local/remote file systems

---

## Calculation of volumes from T1 sequences

First let's define some paths and get FSL loaded etc.

# Setup
##### python cell
    1. enter project ID
    2. If this project doesn't exist in in the temporary space, create it
    3. Set importnat paths that will be used
    4. Make a setup script to save bash variables in
    
    
* Have cluster mounted

In [1]:
import os
from os.path import expanduser
home = expanduser("~")

# Define project name
project = "ssbv" 

# Set project dir in ephermeral 
directory = (home + "/hpc/eph/" + project + "/data/")
if not os.path.exists(directory):
    os.makedirs(directory)
    
workingDir = (home + "/hpc/eph/" + project + "/")
wd = (home + "/eph/" + project + "/")
setup = (workingDir + "/setup.sh" )


##### bash cell
This cell adds bash variables you want to save to a setup script, which can then be called in future python cells. 

1. Add project name
2. Add paths of interest
3. Define modules that will be needed

### Symbolic links can be created to specific shared folders with appropriate permissions, but not top level project directories.

In [2]:
%%writefile $setup

$(: Project label) 
project="ssbv"

$(: dependencies)
export repo=pwd
export dep=${repo}/lib
export workingDir=~/hpc/eph/${project}


wd=/rds/general/user/nbourke/ephemeral/${project}
# imData is a symbolic link to shared directory
imData=~/hpc/imData
raw=${imData}/raw

source ~/.activate_imperial_rcs_login


Overwriting /Users/nbourke/hpc/eph/ssbv//setup.sh


### Check setup

In [3]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir
pwd

echo ${ask}

/Users/nbourke/hpc/eph/ssbv
/Users/nbourke/hpc/repos/ssbv
/tmp/ask


## Copy data

Copy data in BIDS format from source directory into a temporary working directory

#### POPPY scans have not copied across - Not in sourcedata BIDS?

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir
# ----------------


for sub in `cat cifid.txt`; 
    do
    for ses in `ls ${imData}/sourcedata/sub-${sub}`;
        do
        #echo ${ses}
        mkdir -p ${workingDir}/data/sub-${sub}/${ses}/anat/T1w/
        tmp=`ls ${imData}/sourcedata/sub-${sub}/${ses}/anat/T1w/*T1w*`
        echo $tmp
        cp ${tmp} ${workingDir}/data/sub-${sub}/${ses}/anat/T1w/
        gunzip ${workingDir}/data/sub-${sub}/${ses}/anat/T1w/*
    done        
done


- Dry run complete, 159 unique individuals pulled across  
- Data validation is required to check for new/missed participants not copied and to control for individuals with multiple scanning sessions

### Get a list of T1 scans

In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}

# module load fsl
#----------------------------

# setup log dir
if [ ! -d ${workingDir}/commandLogs/ ]; then
    echo "Making log dir!"
    mkdir ${workingDir}/commandLogs/
fi
#

# Make a list of all T1 scans
echo -n "" > ./tmp/t1_list.txt
for s in `ls -d ${workingDir}/data/*`;
    do
    subj=`basename ${s}`
    for ses in `ls ${workingDir}/data/${subj}`; 
        do 
        echo "${wd}"/data/${subj}/${ses}/anat/T1w/${subj}_${ses}_T1w.nii >> ./tmp/t1_list.txt
    done
done

# Step 1. Run segmentation jobs

In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}

# module load fsl
#----------------------------
dep=/rds/general/project/c3nl_shared/live/dependencies

# Segment T1 data
echo -n "" > ./tmp/segmentationJobs.txt
job=./tmp/segmentationJobs.txt
#cjob=${wd}/commandLogs/segmentationJobs.txt

for subject in `cat ./tmp/t1_list.txt`
    do
    echo "${dep}/hpcwrapmatlab.sh \"maxNumCompThreads(3); segment_t1('${subject}');\"" >> ${job}  
done;

    # Run job
    ~/hpc/repos/ssbv/lib/localSubmit ${job} 01:00:00 3 6Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"; head ${job}  

#### Optional: check on your job progress
- wont work locally, need to be on RCS

In [ ]:
!qstat -f

##### After the job has completed look at the output:

Four each subject you should have the following files:
* subject....nii - this is the original untouched nifti - we could later delete it from here as it is stored in the sourcedata folder, in order to save space 
* c1 ....   - this is the grey matter segmented output
* c2 ....   - this is the white matter segmented output
* c3 ....   - this is the CSF segmented output
* rc1 ... rc2  etc.. - this is a rigidly aligned GM segmented output (useful for later when we want to move files to 'standard space' such as MNI)
* seg8 - has details of the segmentation to save SPM time if the software needs to reference the files later on

And most importantly:
* ...... vols.txt - this has your tissue volumes in it


### Vital step: Have a look at your scans to make sure the segmentation has worked properly in each case
We can even generate some commands for you to use in the terminal with FSL.

These are designed so it will be as painless as possible. Load up the terminal, connect to the HPC, make sure you do module load fsl

1. Copy this cell into terminal to run all subjects (quit by pressing ctrl+c in terminal)
2. Run this cell to get commands to copy into terminal to run one at a time

*You want to ensure that the wm and gm are separated nicely and in a way which you think is appropriate.


In [ ]:

# fn = dir('/rds/general/user/nbourke/ephemeral/fa/*.gz');

# % Now we want to view as a movie for QA purposes. 
# figure;ax = gca;
# % use the following to force the Current figure handle to appear outside the live script
# set(gcf,'Visible','on')
# for ii=1:numel(fn)
#     plotNifti([fn(ii).folder,filesep,fn(ii).name],ax);
#     drawnow % will tell Matlab to create animation
#     pause(0.2) % how long to pause between loading the next image
# end

### We can bundle up all of the vols.txt files into a big CSV for convenience, and put this in your notebook folder (within a subfolder called volumetric_results)

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir
#------------------

if [ -d ${workingDir}/volumetric_results ]   
    then
    echo "results folder ready";
    else
    mkdir -p ${workingDir}/volumetric_results
    echo "results folder made";
fi
   
    echo "subject,gm_vol,wm_vol,csf_vol" > ${workingDir}/volumetric_results/volumes.csv 
    for subject in `ls ${workingDir}/data/`
        do
        for ses in `ls ${workingDir}/data/${subject}/`; 
            do 
            volsfile=${workingDir}/data/$subject/${ses}/anat/T1w/*_vols.txt       
            if [ -f ${volsfile} ]
               then
               echo -n "${ses}" >> ${workingDir}/volumetric_results/volumes.csv;
               tail -n 1 ${volsfile} >> ${workingDir}/volumetric_results/volumes.csv;
            fi
        done
    done;
    
echo "done";

#### Next steps

You could download these CSVs into the analysis package of your choice and do some comparisons using the summary measures.

Eg. t-test comparing the GM volume in patients versus controls

## Voxelwise statistics 
### (and steps to get the images in standard space to facilitate this)
In order to do comparisons on the shapes of different brains they need to be moved into 'standard space' such as MNI. SPM can do this for us using 'DARTEL', an approach which preserves volume information on moving.

Then we can use FSL randomise to do voxelwise comparisons between groups

---

## Step 2. First make a 'study-specific' template
This is an average image of your subjects. Rather than going straight to standard space like MNI, it's better to go via a template. You could use all your subjects for this, or a selection of them. Ideally it should be 50% patients 50% controls.  Run the next cell to make a file listing which subjects to use. 

In [ ]:
%%file subjects_for_volumetric_template.txt
sub-control001
sub-control002
sub-control003
sub-patient001
sub-patient002
sub-patient003

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir
#------------------

   
for subject in `ls ${workingDir}/data/`
    do
    for ses in `ls ${workingDir}/data/${subject}/`; 
        do 
        volsfile=${workingDir}/data/$subject/${ses}/anat/T1w/*_vols.txt       
        if [ -f ${volsfile} ]
           then
        
           echo "${subject}" >> ${workingDir}/existingT1.txt;
        fi
    done
done;
    
sort existingT1.txt | uniq -u >> T1_list.txt


Now let's make the template.

This can take a while so you can increase the number from 3 hours to something more generous if you've got lots of subjects.

It will not work if there are too many participants (limit to no more than 100 max)

\* Make a template for each age bin

In [ ]:
# %%bash -s "$setup"
# export setup=$1;
# source ${setup}
# echo ${workingDir}
# #------------------

# # module load fsl
# #----------------------------
# dep=/rds/general/project/c3nl_shared/live/dependencies

# # this uses the RC rigidly aligned files from the segmentation output : if you want to have a look at them use this in bash  
# echo -n "" > ./tmp/templateJob.txt
# job=./tmp/templateJob.txt

#     files=""
  
# echo "" > ./tmp/subj.txt   

# for subject in `ls ${workingDir}/data/`
#     do
#     for ses in `ls ${workingDir}/data/${subject}/`; 
#         do 
#          echo ${subject} >> ./tmp/subj.txt   
#         trc1=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/rc1*T1w.nii`;
#         trc2=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/rc2*T1w.nii`;
#         trc3=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/rc3*T1w.nii`;
#         rc1=${wd}/data/${subject}/${ses}/anat/T1w/${trc1}
#         rc2=${wd}/data/${subject}/${ses}/anat/T1w/${trc2}
#         rc3=${wd}/data/${subject}/${ses}/anat/T1w/${trc3}
        
#         if [ -z "${files}" ]; then
#          files="'${rc1}','${rc2}','${rc3}'";
#         else
#          files="${files},'${rc1}','${rc2}','${rc3}'";
#         fi
#     done
# done
     
#      echo "${dep}/hpcwrapmatlab.sh \"maxNumCompThreads(3); make_template('Template', ${files});\"" > ${job};
              

#     # Setup SSH connection (add to profile)

#     # Run job
#     ~/hpc/repos/ssbv/lib/localSubmit ${job} 71:00:00 3 8Gb
#     echo ""; echo "***"; echo ""; echo "Submitted commands:"; head ${job}  

# 1st session for each subject
* T1_list.txt is a list of unique ids that have existing T1s

In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
#------------------

# module load fsl
#----------------------------
dep=/rds/general/project/c3nl_shared/live/dependencies

# this uses the RC rigidly aligned files from the segmentation output : if you want to have a look at them use this in bash  
echo -n "" > ./tmp/templateJob.txt
job=./tmp/templateJob.txt

files=""
  
for subject in `cat ${workingDir}/T1_list.txt`;
    do
    visit=`basename ${workingDir}/data/${subject}/*` #(*)
    #echo "${files[0]}"
    ses="${visit[0]}"

    trc1=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/rc1*T1w.nii`;
    trc2=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/rc2*T1w.nii`;
    trc3=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/rc3*T1w.nii`;
    rc1=${wd}/data/${subject}/${ses}/anat/T1w/${trc1}
    rc2=${wd}/data/${subject}/${ses}/anat/T1w/${trc2}
    rc3=${wd}/data/${subject}/${ses}/anat/T1w/${trc3}
    
    if [ -z "${files}" ]; then
     files="'${rc1}','${rc2}','${rc3}'";
    else
     files="${files},'${rc1}','${rc2}','${rc3}'";
    fi
done
     
     echo "${dep}/hpcwrapmatlab.sh \"maxNumCompThreads(3); make_template('Template', ${files});\"" > ${job};
              

    # Setup SSH connection (added to profile: ~/.activate_imperial_rcs_login)

    # Run job
    ~/hpc/repos/ssbv/lib/localSubmit ${job} 71:00:00 3 8Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"; head ${job} 

When the job is done, you need to move the completed template files to a nice new folder, as by default they are dumped into the **first** subject's folder 


In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
#------------------

# line=$(head -n 2 ./tmp/subj.txt) 
# echo $line

#mkdir -p ${workingDir}/DARTEL_template/
cp ${workingDir}/data/sub-CIF0354/ses-DREAM003_baseline_v1/anat/T1w/Template_* ${workingDir}/DARTEL_template/;


## Step 3. Make flowfields to the newly made group template for each subject's scan

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir
#------------------

dep=/rds/general/project/c3nl_shared/live/dependencies
    
template_basename="Template"
template=${wd}/DARTEL_template/${template_basename}

unset files;
echo "" > ./tmp/flowfields.txt
job=./tmp/flowfields.txt

for subject in `ls ${workingDir}/data/`
    do
    for ses in `ls ${workingDir}/data/${subject}/`; 
        do 

        files=""
    
        trc1=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/rc1*T1w.nii`;
        trc2=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/rc2*T1w.nii`;
        trc3=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/rc3*T1w.nii`;
        rc1=${wd}/data/${subject}/${ses}/anat/T1w/${trc1}
        rc2=${wd}/data/${subject}/${ses}/anat/T1w/${trc2}
        rc3=${wd}/data/${subject}/${ses}/anat/T1w/${trc3}
        
        files="'${rc1}','${rc2}','${rc3}'";

        if [ -f ${workingDir}/data/${subject}/${ses}/anat/T1w/$trc1 ];
        then

        echo "${dep}/hpcwrapmatlab.sh \"generate_flowfields('${template}', ${files})\"" >> ${job}; 

        else
        echo "No rc1 file for ${subject} at visit ${visit}";
        fi

        unset files;
    done
done
         
    # Run job
    ~/hpc/repos/ssbv/lib/localSubmit ${job} 48:00:00 3 6Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"; head ${job}  

## Step 4. Use the flowfields to send the images to MNI space

This uses smoothing with an 8mm gaussian kernel. This is reasonable...
It uses the 'preserve volumes' option, whereby when a voxel is grown/expanded in the move to MNI, its value its reduced  (ie. the concentration of that voxel is modulated).

If you need to change these settings edit the script move_to_mni.m and then re run your cell.



In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
#------------------
   
dep=/rds/general/project/c3nl_shared/live/dependencies
    
template_basename="Template"
template=${wd}/DARTEL_template/${template_basename}_6.nii


unset files;
echo "" > ./tmp/register2MNI.txt;
job=./tmp/register2MNI.txt;


for subject in `ls ${workingDir}/data/`
    do
    for ses in `ls ${workingDir}/data/${subject}/`; 
        do 

        files=""

        tu_rc1=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/u_rc1*_T1w.nii`;
        tc1=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/c1*_T1w.nii`;
        tc2=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/c2*_T1w.nii`;
        tc3=`basename ${workingDir}/data/${subject}/${ses}/anat/T1w/c3*_T1w.nii`;
        
        u_rc1=${wd}/data/${subject}/${ses}/anat/T1w/${tu_rc1}
        c1=${wd}/data/${subject}/${ses}/anat/T1w/${tc1}
        c2=${wd}/data/${subject}/${ses}/anat/T1w/${tc2}
        c3=${wd}/data/${subject}/${ses}/anat/T1w/${tc3}
        
        files="'${u_rc1}','${c1}','${c2}','${c3}'";


        if [ -f ${workingDir}/data/${subject}/${ses}/anat/T1w/$tu_rc1 ];
        then
        echo "${dep}/hpcwrapmatlab.sh \"move_to_mni('${template}', ${files})\"" >> ${job};
        else
        echo "No flowfield for this person ${subject} and timepoint ${session}";
        fi

        unset files;
    done
done
         
    # Run job
    ~/hpc/repos/ssbv/lib/localSubmit ${job} 48:00:00 3 6Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"; head ${job}
        

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $workingDir
${fsl}
#----------------

## QC CHECK: Have a look at your MNI space spatially normalised and smoothed Jacobian images

### Step 5. Some tract stats in fsl

In [4]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
${fsl}
#----------------

dataDir=${workingDir}/volumetric_results
cdir=${wd}/volumetric_results
mkdir -p ${dataDir}/tractStats
tractDIR=~/hpc/templates/Corrected_Tracts_MNI1mm/

# List standard GM
echo "" > ${workingDir}/smwc1_list.txt

for subject in `ls ${workingDir}/data/`
    do
    for ses in `ls ${workingDir}/data/${subject}/`; 
        do 
        FILE=`ls ${workingDir}/data/${subject}/${ses}/anat/T1w/smwc1*_T1w.nii`  
        if [ -f "$FILE" ]; then
          echo "${wd}"/data/${subject}/${ses}/anat/T1w/smwc1${subject}_${ses}_T1w.nii >> ${workingDir}/smwc1_list.txt
        fi
    done
done
    

# list standard WM
echo "" > ${workingDir}/smwc2_list.txt
for subject in `ls ${workingDir}/data/`
    do
    for ses in `ls ${workingDir}/data/${subject}/`; 
        do 
        FILE=`ls ${workingDir}/data/${subject}/${ses}/anat/T1w/smwc2*_T1w.nii`  
        if [ -f "$FILE" ]; then
          echo "${wd}"/data/${subject}/${ses}/anat/T1w/smwc2${subject}_${ses}_T1w.nii >> ${workingDir}/smwc2_list.txt
        fi
    done
done
    
echo "" > ./tmp/smwc1statsJob.txt;
job1=./tmp/smwc1statsJob.txt;

echo "" > ./tmp/smwc2statsJob.txt;
job2=./tmp/smwc2statsJob.txt;
    
echo "module load fsl; fslmerge -t ${cdir}/all_smwc1 \`cat ${wd}/smwc1_list.txt\`; fslmaths ${cdir}/all_smwc1 -max 0 -Tmin -bin ${cdir}/all_smwc1_mask -odt char; fslmaths ${cdir}/all_smwc1 -mas ${cdir}/all_smwc1_mask ${cdir}/all_smwc1; fslmaths ${cdir}/all_smwc1 -Tmean ${cdir}/mean_smwc1; fslmaths ${cdir}/mean_smwc1 -thr 0.5 -bin ${cdir}/mean_smwc1_mask " > ${job1}
echo "module load fsl; fslmerge -t ${cdir}/all_smwc2 \`cat ${wd}/smwc2_list.txt\`; fslmaths ${cdir}/all_smwc2 -max 0 -Tmin -bin ${cdir}/all_smwc2_mask -odt char; fslmaths ${cdir}/all_smwc2 -mas ${cdir}/all_smwc2_mask ${cdir}/all_smwc2; fslmaths ${cdir}/all_smwc2 -Tmean ${cdir}/mean_smwc2; fslmaths ${cdir}/mean_smwc2 -thr 0.5 -bin ${cdir}/mean_smwc2_mask " > ${job2} 

# # create mean GM
# fslmaths ${dataDir}/all_smwc1 -max 0 -Tmin -bin ${dataDir}/all_smwc1_mask -odt char
# fslmaths ${dataDir}/all_smwc1 -mas ${dataDir}/all_smwc1_mask ${dataDir}/all_smwc1
# fslmaths ${dataDir}/all_smwc1 -Tmean ${dataDir}/mean_smwc1
# fslmaths ${dataDir}/mean_smwc1 -thr 0.5 -bin ${dataDir}/mean_smwc1_mask 

# # create mean WM
# fslmaths ${dataDir}/all_smwc2 -max 0 -Tmin -bin ${dataDir}/all_smwc2_mask -odt char
# fslmaths ${dataDir}/all_smwc2 -mas ${dataDir}/all_smwc2_mask ${dataDir}/all_smwc2
# fslmaths ${dataDir}/all_smwc2 -Tmean ${dataDir}/mean_smwc2
# fslmaths ${dataDir}/mean_smwc2 -thr 0.5 -bin ${dataDir}/mean_smwc2_mask 

    # Run job
    ~/hpc/repos/ssbv/lib/localSubmit ${job1} 48:00:00 3 16Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"; head ${job1}  
    
        # Run job
    ~/hpc/repos/ssbv/lib/localSubmit ${job2} 48:00:00 3 16Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"; head ${job2}  


/Users/nbourke/hpc/eph/ssbv
input is = ./tmp/smwc1statsJob.txt
Walltime = 48:00:00
Number of CPUs = 3
Memory = 16Gb
login is : ssh nbourke@login.hpc.ic.ac.uk
jobs =        1
Check this correct
PBS file = 
/var/folders/kw/ttpm9tz913j6zqhngnlnfsg00000gn/T/tmp.nsBrQYPV
Job submitted: Tue 29 Mar 2022 11:12:51 BST
5378918.pbs

***

Submitted commands:
module load fsl; fslmerge -t /rds/general/user/nbourke/ephemeral/ssbv/volumetric_results/all_smwc1 `cat /rds/general/user/nbourke/ephemeral/ssbv/smwc1_list.txt`; fslmaths /rds/general/user/nbourke/ephemeral/ssbv/volumetric_results/all_smwc1 -max 0 -Tmin -bin /rds/general/user/nbourke/ephemeral/ssbv/volumetric_results/all_smwc1_mask -odt char; fslmaths /rds/general/user/nbourke/ephemeral/ssbv/volumetric_results/all_smwc1 -mas /rds/general/user/nbourke/ephemeral/ssbv/volumetric_results/all_smwc1_mask /rds/general/user/nbourke/ephemeral/ssbv/volumetric_results/all_smwc1; fslmaths /rds/general/user/nbourke/ephemeral/ssbv/volumetric_results/all_smw

ls: /Users/nbourke/hpc/eph/ssbv/data/sub-CIF0092/ses-2013-10-28-13_10_28_visit1_SMH0076/anat/T1w/smwc1*_T1w.nii: No such file or directory
ls: /Users/nbourke/hpc/eph/ssbv/data/sub-CIF0092/ses-2015-10-27-POPPY_SMH0076/anat/T1w/smwc1*_T1w.nii: No such file or directory
ls: /Users/nbourke/hpc/eph/ssbv/data/sub-CIF0117/ses-2013-11-29-13_11_29_visit1_SMH0070/anat/T1w/smwc1*_T1w.nii: No such file or directory
ls: /Users/nbourke/hpc/eph/ssbv/data/sub-CIF0117/ses-2015-10-13-POPPY_SMH0070_MV2/anat/T1w/smwc1*_T1w.nii: No such file or directory
ls: /Users/nbourke/hpc/eph/ssbv/data/sub-CIF0135/ses-2013-12-20-13_12_20_visit1_SMH0193/anat/T1w/smwc1*_T1w.nii: No such file or directory
ls: /Users/nbourke/hpc/eph/ssbv/data/sub-CIF0135/ses-2015-10-30-POPPY_SMH0193_V2/anat/T1w/smwc1*_T1w.nii: No such file or directory
ls: /Users/nbourke/hpc/eph/ssbv/data/sub-CIF0148/ses-2014-01-21-14_01_21_visit1_SMH0228v1/anat/T1w/smwc1*_T1w.nii: No such file or directory
ls: /Users/nbourke/hpc/eph/ssbv/data/sub-CIF0148

In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
${fsl}
#----------------

dataDir=${workingDir}/volumetric_results
cdir=${wd}/volumetric_results
mkdir -p ${dataDir}/tractStats
tractDIR=~/hpc/templates/Corrected_Tracts_MNI1mm/

#Find the tract masks
cd ${tractDIR}
TRACTS=`ls *.gz`

# Nested for loop - for each mask and each metric do fslstats
for i in $TRACTS; 
do
    #j=$(echo ${i} | cut -d '_' -f2-)
    k=$(echo ${i} | cut -d '.' -f1)
    
    echo $k > ${dataDir}/tractStats/vol_MNI_${k}.txt
    fslstats -t ${dataDir}/all_smwc2.nii.gz -k ${tractDIR}/$i -M >> ${dataDir}/tractStats/vol_MNI_${k}.txt
done 

cp ${workingDir}/smwc2_list.txt ${dataDir}/tractStats/aaa.txt

echo "WM_VOL" > ${dataDir}/tractStats/vol_MNI_WM.txt
fslstats -t ${dataDir}/all_smwc2.nii.gz -k ${dataDir}/mean_smwc2_mask.nii.gz -M >> ${dataDir}/tractStats/vol_MNI_WM.txt 
echo "GM_VOL" > ${dataDir}/tractStats/vol_MNI_GM.txt
fslstats -t ${dataDir}/all_smwc1.nii.gz -k ${dataDir}/mean_smwc1_mask.nii.gz -M >> ${dataDir}/tractStats/vol_MNI_GM.txt


# paste -d , `ls` >> ../tractStats.csv

# Grey matter ROIs (Harvard-Oxford atlas)

In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}

workingDir=/Users/nbourke/hpc/eph/ssbv/
#------------------


# set output dir
dataDir=${workingDir}/volumetric_results
mkdir -p ${dataDir}/roiStats

# source atlas
atlasDIR=~/hpc/templates/HarvardOxford-atlas/

# set job
for brain in cortical subcortical; 
    do
    cd ${atlasDIR}/${brain}/masks/
    ROI=`ls *nii.gz`

    # Nested for loop - for each mask and each metric do fslstats
    for i in ${ROI}; 
        do
        k=$(echo ${i} | cut -d '.' -f1); 
        echo $k > ${dataDir}/roiStats/vol_MNI_${k}.txt;
        #echo "fslstats -t ${dataDir}/all_smwc1.nii.gz -k ${atlasDIR}/${brain}/masks/${i} -M -V | awk {'print \$1 * \$3'} >> ${dataDir}/roiStats/vol_MNI_${k}.txt" >> ${job}  
        fslstats -t ${dataDir}/all_smwc1.nii.gz -k ${atlasDIR}/${brain}/masks/${i} -M >> ${dataDir}/roiStats/MI_MNI_${k}.txt
    done
done 
        

# *Remember to copy relevent outputs to derivitives! 

### Additional ROIs

In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
${fsl}
#----------------

dataDir=${workingDir}/volumetric_results
mkdir -p ${dataDir}/tractStats
tractDIR=~/templates/Corrected_Tracts/


#Find the tract masks
cd ${tractDIR}
TRACTS=`ls *.gz`

# Nested for loop - for each mask and each metric do fslstats
for i in $TRACTS; 
do
    #j=$(echo ${i} | cut -d '_' -f2-)
    k=$(echo ${i} | cut -d '.' -f1)
    
    echo $k > ${dataDir}/tractStats/vol_MNI_${k}.txt
    fslstats -t ${dataDir}/all_smwc2.nii.gz -k ${tractDIR}/$i -M >> ${dataDir}/tractStats/vol_MNI_${k}.txt
done 

cp ${workingDir}/smwc2_list.txt ${dataDir}/tractStats/aaa.txt


# paste -d , `ls` >> ../tractStats.csv

## Voxelwise analysis

In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
${fsl}
#----------------

# Update working dir, as done after main analysis was conducted
wd=/rds/general/project/c3nl_djs_imaging_data/live/analysis/paeds

# Remove pilot
fslmerge -t ${wd}/derivatives/volumetric_results/all_smwc1 `cat ${wd}/derivatives/volumetric_results/smwc1_list.txt`
fslmerge -t ${wd}/derivatives/volumetric_results/all_smwc2 `cat ${wd}/derivatives/volumetric_results/smwc2_list.txt`

# design=${scriptDir}/scripts/design/demo.mat
# contrast=${scriptDir}/scripts/design/mainContrast.con 
# setup_masks ${design} ${contrast} ${scriptDir}/scripts/design/${ii} `cat ${scriptDir}/scripts/design/masks.txt` 


In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
module load fsl/6.0.1/
#----------------------------

wd=/rds/general/project/c3nl_djs_imaging_data/live/analysis/paeds
design=${wd}/scripts/design/demo.mat
contrast=${wd}/scripts/design/mainContrast.con

# Run setup masks command
setup_masks ${design} ${contrast} ${wd}/scripts/design/MNIlesion `ls ${wd}/derivatives/lesionMasks/MNI/bin*`   


In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
module load fsl/6.0.1/
#----------------------------

# Update working dir, as done after main analysis was conducted
wd=/rds/general/project/c3nl_djs_imaging_data/live/analysis/paeds
output=${wd}/tbss_lesion_output/

# Remove pilot
# fslmerge -t ${wd}/derivatives/volumetric_results/all_smwc1 `cat ${wd}/derivatives/volumetric_results/smwc1_list.txt`
# fslmerge -t ${wd}/derivatives/volumetric_results/all_smwc2 `cat ${wd}/derivatives/volumetric_results/smwc2_list.txt`

for ii in smwc1 smwc2
    do
    data_input=${wd}/derivatives/volumetric_results/all_${ii}.nii.gz
    data_mask=${wd}/derivatives/volumetric_results/mean_${ii}_mask.nii.gz
    design=${wd}/scripts/design/MNIlesion.mat
    contrast=${wd}/scripts/design/MNIlesion.con
    basename=${wd}/scripts/design/MNIlesion.nii.gz
    
    mkdir ${output}/${ii}
    ## Run command ##
    ${dep}/pbs_randomise_par -wt 24:00:00 -mem 14Gb -i ${data_input} -o ${output}/${ii}/${ii}_TBSS -m ${data_mask} -d ${design} -t ${contrast} --vxl=-4 --vxf=${basename} -n 5000 --T2 -V  

done

# Freesurfer

In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}

module load fsl
#----------------------------

# Set job command File 
echo -n "" > ${workingDir}/${project}_fs_job.txt
job=${workingDir}/${project}_fs_job.txt


### Job loop ###
for subject in sub-CIF1703 sub-CIF2178 #`ls ${workingDir}/data/`
    do
    for ses in `ls ${workingDir}/data/${subject}/`; 
        do 
        rm -r ${workingDir}/data/${subject}/${ses}/anat/T1w/fs
        rm -r ${workingDir}/data/${subject}/${ses}/anat/T1w/fsaverage
        rm -r ${workingDir}/data/${subject}/${ses}/anat/T1w/T1w
        echo "/rds/general/user/nbourke/home/group_paeds/scripts/pbsFreesurfer -i ${subject} ${ses} ${workingDir}" >> ${job}           
    done
done

    # Run job
    ${dep}/hpcSubmit ${job} 20:00:00 1 12Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}
     

## *The following aparcstats2table command works when copied into the terminal

In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}

module load fsl
module load freesurfer
#----------------------------
EXPERIMENT_DIR=${workingDir}
export SUBJECTS_DIR=`${workingDir}/data/`


counter=1
### Job loop ###
for subject in `ls ${workingDir}/data/`
    do
    for ses in `ls ${workingDir}/data/${subject}/`; 
        do  
        echo -e "$( if [ "${counter}" -eq "1" ]; then echo "First run: "; fi )${subject}"

        
        XX=${workingDir}/data/${subject}/${ses}/anat/T1w/fs 
        #Thickness
        aparcstats2table --subjects $XX --hemi rh --meas thickness  --tablefile ${workingDir}/data/${subject}/${ses}/anat/T1w/fs/rh_thick_aparc_stats.txt
        aparcstats2table --subjects $XX --hemi lh --meas thickness  --tablefile ${workingDir}/data/${subject}/${ses}/anat/T1w/fs/lh_thick_aparc_stats.txt
        # Volume
        aparcstats2table --subjects $XX --hemi rh --meas volume --tablefile ${workingDir}/data/${subject}/${ses}/anat/T1w/fs/rh_vol_aparc_stats.txt
        aparcstats2table --subjects $XX --hemi lh --meas volume --tablefile ${workingDir}/data/${subject}/${ses}/anat/T1w/fs/lh_vol_aparc_stats.txt
        
        
    done
    counter=$((counter +1))
done


### Extracting FreeSurfer measurments

In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}

#----------------------------

echo "" > ${workingDir}/rh_thick_aparc_stats.txt
echo "" > ${workingDir}/lh_thick_aparc_stats.txt
echo "" > ${workingDir}/rh_vol_aparc_stats.txt
echo "" > ${workingDir}/lh_vol_aparc_stats.txt

counter=1
### Job loop ###
for subject in `ls ${workingDir}/data/`
    do
    for ses in `ls ${workingDir}/data/${subject}/`; 
        do  
        XX=${workingDir}/data/${subject}/${ses}/anat/T1w/fs
        
        echo -e "$( if [ "${counter}" -eq "10" ]; then sed '1q;d' ${XX}/rh_thick_aparc_stats.txt >> ${workingDir}/rh_thick_aparc_stats.txt; sed '1q;d' ${XX}/lh_thick_aparc_stats.txt >> ${workingDir}/lh_thick_aparc_stats.txt; sed '1q;d' ${XX}/rh_vol_aparc_stats.txt >> ${workingDir}/rh_vol_aparc_stats.txt; sed '1q;d' ${XX}/lh_vol_aparc_stats.txt >> ${workingDir}/lh_vol_aparc_stats.txt; fi )"  
 
        sed '2q;d' ${XX}/rh_thick_aparc_stats.txt >> ${workingDir}/rh_thick_aparc_stats.txt
        sed '2q;d' ${XX}/lh_thick_aparc_stats.txt >> ${workingDir}/lh_thick_aparc_stats.txt
        sed '2q;d' ${XX}/rh_vol_aparc_stats.txt >> ${workingDir}/rh_vol_aparc_stats.txt
        sed '2q;d' ${XX}/lh_vol_aparc_stats.txt >> ${workingDir}/lh_vol_aparc_stats.txt

    done
    counter=$((counter +1))
done


1. mv to local
2. open in excell
3. correct heading
4. save as csv

### Registering lesion masks to MNI

In [ ]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}

#----------------------------

echo "" > ${workingDir}/commandLogs/lesion_reg.txt
job=${workingDir}/commandLogs/lesion_reg.txt
mkdir ${workingDir}/tmpReg
### Job loop ###
for subject in `ls ${workingDir}/data/`
    do
    for ses in `ls ${workingDir}/data/${subject}/`; 
        do  
        brain=${workingDir}/data/${subject}/${ses}/anat/T1w/${subject}_${ses}_T1w_brain.nii.gz  
        lesion=${workingDir}/lesionMasks/${subject}_${ses}_contusion.nii.gz
        
        if [ -f "$lesion" ]; then
            echo "$ses has a lesion"
            echo "${fsl}; flirt -in ${brain} -ref /rds/general/apps/fsl/5.0.10/install/data/standard/MNI152_T1_1mm_brain.nii.gz -omat ${workingDir}/tmpReg/${subject}_${ses}_T1brain2MNI.mat -dof 6 -cost mutualinfo -searchcost mutualinfo; flirt -in ${lesion} -ref /rds/general/apps/fsl/5.0.10/install/data/standard/MNI152_T1_1mm_brain.nii.gz -applyxfm -init ${workingDir}/tmpReg/${subject}_${ses}_T1brain2MNI.mat -out ${workingDir}/lesionMasks/MNI/${subject}_${ses}_contusion_MNI.nii.gz" >> ${job}  
        else
            echo "$ses does not have a lesion, making empty mask file"
            cp /rds/general/user/nbourke/home/templates/MNI152_T1_1mm_empty_mask.nii ${workingDir}/lesionMasks/MNI/${subject}_${ses}_empty_mask_MNI.nii.gz
        fi
    done
done

    # Run job
    ${dep}/hpcSubmit ${job} 08:00:00 1 12Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}